In [1]:
# Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

C:\Users\Admin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
# Loading the dataset

In [6]:
email_data = pd.read_csv(r"C:\Users\Admin\Downloads\ham_spam.csv", encoding="latin")

In [7]:
email_data.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or å£10,000..."
4,spam,okmail: Dear Dave this is your final notice to...


In [8]:
# Cleaning the data

In [9]:
import re

In [11]:
stop_words = []
with open("D:\\Data science\\Python codes\\Data sets\\NLP words list\\stopwords.txt") as f:
    stop_words = f.read()

In [12]:
# splitting the entire string by giving separator as "\n" to get list of all stop words

In [13]:
stop_words = stop_words.split("\n")

In [14]:
stop_words

['#',
 '#  stopwords.txt',
 '#',
 '#  Freely available stopword list, balancing coverage and size.',
 '#',
 '#  From http://www.lextek.com/manuals/onix/stopwords1.html',
 'a',
 'about',
 'above',
 'across',
 'after',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'among',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyone',
 'anything',
 'anywhere',
 'are',
 'area',
 'areas',
 'around',
 'as',
 'ask',
 'asked',
 'asking',
 'asks',
 'at',
 'away',
 'b',
 'back',
 'backed',
 'backing',
 'backs',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'been',
 'before',
 'began',
 'behind',
 'being',
 'beings',
 'best',
 'better',
 'between',
 'big',
 'both',
 'but',
 'by',
 'c',
 'came',
 'can',
 'cannot',
 'case',
 'cases',
 'certain',
 'certainly',
 'clear',
 'clearly',
 'come',
 'could',
 'd',
 'did',
 'differ',
 'different',
 'differently',
 'do',
 'does',
 'done',
 'down',
 'down',
 'downed',
 'downing',
 'downs',
 'duri

In [15]:
# Defining cleaning text function

In [16]:
def cleaning_text(i):
    i = re.sub("[^A-Za-z" "]+"," ",i).lower()
    i = re.sub("[0-9" "]+"," ",i)
    w = []
    for word in i.split(" "):
        if len(word) > 3:
            w.append(word)
    return (" ".join(w))

In [19]:
# testing above function with sample text

In [20]:
cleaning_text("Hope you are having a good week. Just checking in")

'hope having good week just checking'

In [21]:
cleaning_text("hope i can understand your feelings 123121. 123 hi how .. are you?")

'hope understand your feelings'

In [18]:
cleaning_text("This is Awsome 1231312 $#%$# a i he yu nwj")

'this awsome'

#### Function is working properly, hence implementing it onto email data

In [22]:
email_data.text = email_data.text.apply(cleaning_text)

In [24]:
email_data.text.head()

0                  hope having good week just checking
1                                     give back thanks
2                                 also doing only have
3    complimentary star ibiza holiday cash needs yo...
4    okmail dear dave this your final notice collec...
Name: text, dtype: object

In [25]:
# removing empty rows

In [26]:
email_data.shape

(5559, 2)

In [27]:
email_data = email_data.loc[email_data.text !=" ",:]

In [29]:
# creating a matrix of token counts for the entire text document 

In [30]:
def split_into_words(i):
    return [word for word in i.split(" ")]

In [32]:
# Checking the working of function

In [31]:
split_into_words("I am going now")

['I', 'am', 'going', 'now']

In [33]:
# splitting data into train and test data sets 

In [34]:
from sklearn.model_selection import train_test_split

C:\Users\Admin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [35]:
email_train, email_test = train_test_split(email_data, test_size=0.3)

In [36]:
# Preparing email texts into word count matrix format 

In [37]:
emails_bow =CountVectorizer(analyzer=split_into_words).fit(email_data.text)

In [39]:
# For all messages

In [40]:
all_emails_matrix = emails_bow.transform(email_data.text) 

In [42]:
all_emails_matrix.shape

(5559, 6661)

In [43]:
# for training messages

In [45]:
train_emails_matrix = emails_bow.transform(email_train.text)
train_emails_matrix.shape

(3891, 6661)

In [46]:
# for testing messages

In [47]:
test_emails_matrix = emails_bow.transform(email_test.text)
test_emails_matrix.shape

(1668, 6661)

## Building Naive Bayes model

In [48]:
from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB

### Multinomial Naive Bayes

In [49]:
classifier_mb = MB()

In [50]:
classifier_mb.fit(train_emails_matrix, email_train.type)

MultinomialNB()

In [51]:
train_pred_m = classifier_mb.predict(train_emails_matrix)

In [52]:
accuracy_train_m = np.mean(train_pred_m == email_train.type)
accuracy_train_m

0.9884348496530455

In [53]:
test_pred_m = classifier_mb.predict(test_emails_matrix)

In [54]:
accuracy_test_m = np.mean(test_pred_m == email_test.type)
accuracy_test_m

0.9718225419664268

### Gaussian Naive Bayes

In [55]:
classifier_gb = GB()

In [56]:
classifier_gb.fit(train_emails_matrix.toarray(), email_train.type.values)    # we need to convert tfidf into array format which is compatible for gaussian naive bayes

GaussianNB()

In [57]:
train_pred_g = classifier_gb.predict(train_emails_matrix.toarray())

In [61]:
accuracy_train_g = np.mean(train_pred_g==email_train.type)
accuracy_train_g

0.9236700077101002

In [59]:
test_pred_g = classifier_gb.predict(test_emails_matrix.toarray())

In [62]:
accuracy_test_g = np.mean(test_pred_g==email_test.type)
accuracy_test_g

0.8357314148681055

#### Multinomial NB is giving higher accuracy of 97.18 % compared to 83.57% of Gaussian NB, hence Multinomial model is best suited to detect spam text from emails